In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install aif360

In [6]:
# !pip install BlackBoxAuditing==0.1.54

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import RicciDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'one_repair_lvl_many_models_ricci'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
DATASET_NAME = 'RicciDataset'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.4, 0.7]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'ricci_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'd6a4d686-4369-4bca-95c8-7be5d0740b15',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  d6a4d686-4369-4bca-95c8-7be5d0740b15


## Initialize custom objects

In [9]:
data_loader = RicciDataset()
data_loader.X_data.head()

,Oral,Written,Combine,Position,Race
0,89.52,95,92.808,Captain,White
1,80.00,95,89.000,Captain,White
2,82.38,87,85.152,Captain,White
3,88.57,76,81.028,Captain,White
4,76.19,84,80.876,Captain,White


In [10]:
data_loader.y_data.value_counts()

0    62
1    56
Name: Promoted, dtype: int64

In [11]:
data_loader.X_data.shape

(118, 5)

In [12]:
data_loader.X_data['Race'].value_counts()

White        68
Non-White    50
Name: Race, dtype: int64

## Run experiment iterations

### Experiment iteration 1

In [13]:
# tuned_params_filenames = ['tuning_results_Ricci_alpha_0.0_20230806__143119.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [14]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [15]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
#                                    with_tuning=False,
#                                    tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True,
                                   dataset_name=DATASET_NAME)

2023-08-06 16:49:02 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0]',
 'model_init_seed': 100,
 'session_uuid': 'd6a4d686-4369-4bca-95c8-7be5d0740b15'}




2023-08-06 16:49:02 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 84,  85,  37,  45,  97,  26, 101, 112,  21,  33,  11,  46,  96,
             25,  32,  99, 108,  51,  29,  64],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 84,  85,  37,  45,  97,  26, 101, 112,  21,  33,  11,  46,  96,
             25,  32,  99, 108,  51,  29,  64],
           dtype='int64')


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

2023-08-06 16:50:28 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
INFO:root:Successfully tested classifiers by bootstrap
2023-08-06 16:50:28 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics
INFO:root:Successfully computed predict proba metrics



[MLPClassifier] Metrics matrix:


,Metric,overall,Race_priv,Race_dis,Model_Seed,Model_Name,Model_Params
0,Mean,0.685829,0.499715,0.829644,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
1,Std,0.069973,0.139455,0.016282,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
2,IQR,0.082799,0.163163,0.020700,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
3,Aleatoric_Uncertainty,0.048079,0.087151,0.017886,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
4,Overall_Uncertainty,0.148760,0.290915,0.038914,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
5,Statistical_Bias,0.049605,0.098594,0.011749,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
6,Jitter,0.064280,0.126527,0.016181,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
7,Per_Sample_Accuracy,0.954615,0.909412,0.989545,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
8,Label_Stability,0.909231,0.818824,0.979091,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
9,TPR,1.000000,1.000000,1.000000,101,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."


2023-08-06 16:50:30 experiment_interface.py INFO    : Experiment run was successful!
INFO:root:Experiment run was successful!


### Experiment iteration 2

In [30]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Ricci_alpha_0.0_20230806__204940.csv',
    'tuning_results_Ricci_alpha_0.4_20230806__205057.csv',
    'tuning_results_Ricci_alpha_0.7_20230806__205137.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True,
                                   dataset_name=DATASET_NAME)

2023-08-06 16:56:40 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.4, 0.7]',
 'model_init_seed': 200,
 'session_uuid': 'd6a4d686-4369-4bca-95c8-7be5d0740b15'}




2023-08-06 16:56:40 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([72, 66, 34, 21, 86, 9, 13, 36, 98, 2, 67, 10, 87, 110, 12, 46, 48,
            102, 104, 38],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([72, 66, 34, 21, 86, 9, 13, 36, 98, 2, 67, 10, 87, 110, 12, 46, 48,
            102, 104, 38],
           dtype='int64')


Multiple alphas:   0%|          | 0/3 [00:00<?, ?it/s]

...


[LogisticRegression] Metrics matrix:


,Metric,overall,Race_priv,Race_dis,Model_Seed,Model_Name,Model_Params
0,Mean,0.543455,0.401808,0.747074,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
1,Std,0.039544,0.041400,0.036876,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
2,IQR,0.053544,0.056775,0.048898,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
3,Aleatoric_Uncertainty,0.675437,0.701540,0.637913,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
4,Overall_Uncertainty,0.682847,0.709222,0.644935,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
5,Statistical_Bias,0.229713,0.242473,0.211371,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
6,Jitter,0.066672,0.063684,0.070967,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
7,Per_Sample_Accuracy,0.945385,0.959130,0.925625,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
8,Label_Stability,0.902051,0.918261,0.878750,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."
9,TPR,0.900000,1.000000,0.500000,201,LogisticRegression,"{'C': 0.1, 'class_weight': None, 'dual': False..."






##############################  [Model 3 / 4] Analyze RandomForestClassifier  ##############################
Model seed:  201

Protected groups splits:
Race_priv (23, 1)
Race_dis (16, 1)




2023-08-06 16:56:44 abstract_overall_variance_analyzer.py INFO    : Start classifiers testing by bootstrap
INFO:root:Start classifiers testing by bootstrap


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [13]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Ricci_alpha_0.0_20230806__204940.csv',
    'tuning_results_Ricci_alpha_0.4_20230806__205057.csv',
    'tuning_results_Ricci_alpha_0.7_20230806__205137.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [14]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True,
                                   dataset_name=DATASET_NAME)

2023-08-06 16:57:26 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[0.0, 0.4, 0.7]',
 'model_init_seed': 300,
 'session_uuid': 'd6a4d686-4369-4bca-95c8-7be5d0740b15'}




2023-08-06 16:57:26 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 69,  17,  62,  67,   7,  46,  14, 111,  18,  48,  29, 102,  39,
             52,  58,  83,  11,  56, 116, 113],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 69,  17,  62,  67,   7,  46,  14, 111,  18,  48,  29, 102,  39,
             52,  58,  83,  11,  56, 116, 113],
           dtype='int64')


Multiple alphas:   0%|          | 0/3 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

2023-08-06 17:01:33 abstract_overall_variance_analyzer.py INFO    : Successfully tested classifiers by bootstrap
INFO:root:Successfully tested classifiers by bootstrap
2023-08-06 17:01:33 abstract_overall_variance_analyzer.py INFO    : Successfully computed predict proba metrics
INFO:root:Successfully computed predict proba metrics



[MLPClassifier] Metrics matrix:


,Metric,overall,Race_priv,Race_dis,Model_Seed,Model_Name,Model_Params
0,Mean,0.482821,0.427587,0.535293,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
1,Std,0.182353,0.182532,0.182184,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
2,IQR,0.101786,0.096354,0.106946,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
3,Aleatoric_Uncertainty,0.315286,0.308728,0.321516,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
4,Overall_Uncertainty,0.512744,0.509687,0.515649,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
5,Statistical_Bias,0.170421,0.173764,0.167246,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
6,Jitter,0.161113,0.146067,0.175407,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
7,Per_Sample_Accuracy,0.846667,0.854211,0.839500,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
8,Label_Stability,0.794872,0.823158,0.768000,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."
9,TPR,0.900000,0.846154,1.000000,301,MLPClassifier,"{'activation': 'logistic', 'alpha': 0.0001, 'b..."


2023-08-06 17:01:34 experiment_interface.py INFO    : Experiment run was successful!
INFO:root:Experiment run was successful!


### Experiment iteration 4

In [83]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Ricci_alpha_0.0_20230806__204940.csv',
    'tuning_results_Ricci_alpha_0.4_20230806__205057.csv',
    'tuning_results_Ricci_alpha_0.7_20230806__205137.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [84]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True,
                                   dataset_name=DATASET_NAME)

2023-08-06 14:04:55 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, '
                                 '0.8, 0.9, 1.0]',
 'intervention_param': 1.0,
 'model_init_seed': 400,
 'session_uuid': '6d916473-4d0f-4247-9638-b4f140ec4b28'}




2023-08-06 14:04:55 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([101,  24,  35,  10,  93,  89,  83, 108,  48,  47,  59,  58,  85,
             80,  11,  77,  71, 106,  66,  99],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([101,  24,  35,  10,  93,  89,  83, 108,  48,  47,  59,  58,  85,
             80,  11,  77,  71, 106,  66,  99],
           dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  1.0
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_ricci/tuning_results_Ricci_alpha_1.0_20230806__174821.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 401, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [85]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Ricci_alpha_0.0_20230806__204940.csv',
    'tuning_results_Ricci_alpha_0.4_20230806__205057.csv',
    'tuning_results_Ricci_alpha_0.7_20230806__205137.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [86]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 14:10:09 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, '
                                 '0.8, 0.9, 1.0]',
 'intervention_param': 1.0,
 'model_init_seed': 500,
 'session_uuid': '6d916473-4d0f-4247-9638-b4f140ec4b28'}




2023-08-06 14:10:09 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([74, 92, 16, 54, 50, 81, 66, 95, 7, 84, 83, 113, 115, 1, 9, 5, 98,
            109, 26, 103],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([74, 92, 16, 54, 50, 81, 66, 95, 7, 84, 83, 113, 115, 1, 9, 5, 98,
            109, 26, 103],
           dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  1.0
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_ricci/tuning_results_Ricci_alpha_1.0_20230806__174821.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 501, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [87]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Ricci_alpha_0.0_20230806__204940.csv',
    'tuning_results_Ricci_alpha_0.4_20230806__205057.csv',
    'tuning_results_Ricci_alpha_0.7_20230806__205137.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [88]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True, 
                                   dataset_name=DATASET_NAME)

2023-08-06 14:15:23 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, '
                                 '0.8, 0.9, 1.0]',
 'intervention_param': 1.0,
 'model_init_seed': 600,
 'session_uuid': '6d916473-4d0f-4247-9638-b4f140ec4b28'}




2023-08-06 14:15:23 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([102, 112, 115,  24,  57, 109,  61,  56,  80,   3, 104, 103, 111,
             45,  14,  23, 113,   4,  83,  17],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([102, 112, 115,  24,  57, 109,  61,  56,  80,   3, 104, 103, 111,
             45,  14,  23, 113,   4,  83,  17],
           dtype='int64')


Multiple alphas:   0%|          | 0/11 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  1.0
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_ricci/tuning_results_Ricci_alpha_1.0_20230806__174821.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 601, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]